# Snakemake tutorial

Snakefile contents are taken from [Snakemake tutorial page](https://snakemake.readthedocs.io/en/stable/tutorial/basics.html). Tutorial data is available under `data` folder. You can run the files by;

1. **Terminal** : Start a terminal with launcher and you can type/run the commands from tutorial page
2. **Notebook** : To run `bash` commands, instead of `!` you should be using `%%bash` in cells, otherwise conda environment is not recognized

In [1]:
! ls 

binder	     envs	  scripts     Snakefile4     Snakefile-all2
config.yaml  index.ipynb  Snakefile1  Snakefile5     Snakefile-env
dag.svg      LICENSE	  Snakefile2  Snakefile6     Snakefile-wrap
data	     README.md	  Snakefile3  Snakefile-all


In [2]:
! cat Snakefile1

rule bwa_map:
    input:
        "data/genome.fa",
        "data/samples/A.fastq"
    output:
        "mapped_reads/A.bam"
    shell:
        "bwa mem {input} | samtools view -Sb - > {output}"



In [3]:
%%bash

snakemake -np mapped_reads/A.bam -s Snakefile1

Building DAG of jobs...
Job counts:
	count	jobs
	1	bwa_map
	1

[Mon Apr  8 13:42:10 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 0

bwa mem data/genome.fa data/samples/A.fastq | samtools view -Sb - > mapped_reads/A.bam
Job counts:
	count	jobs
	1	bwa_map
	1


In [4]:
%%bash

snakemake -s Snakefile1 mapped_reads/A.bam

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bwa_map
	1

[Mon Apr  8 13:42:16 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 0

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25000 sequences (2525000 bp)...
[M::mem_process_seqs] Processed 25000 reads in 1.328 CPU sec, 1.432 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem data/genome.fa data/samples/A.fastq
[main] Real time: 1.753 sec; CPU: 1.459 sec
[Mon Apr  8 13:42:18 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-04-08T134216.397340.snakemake.log


In [5]:
%%bash

snakemake -s Snakefile1 -f mapped_reads/A.bam

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bwa_map
	1

[Mon Apr  8 13:42:21 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 0

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25000 sequences (2525000 bp)...
[M::mem_process_seqs] Processed 25000 reads in 1.495 CPU sec, 1.508 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem data/genome.fa data/samples/A.fastq
[main] Real time: 1.933 sec; CPU: 1.647 sec
[Mon Apr  8 13:42:23 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-04-08T134221.780203.snakemake.log


In [6]:
! ls mapped_reads/

A.bam


In [7]:
! cat Snakefile2

rule bwa_map:
    input:
        "data/genome.fa",
        "data/samples/{sample}.fastq"
    output:
        "mapped_reads/{sample}.bam"
    shell:
        "bwa mem {input} | samtools view -Sb - > {output}"


In [8]:
%%bash

snakemake -np mapped_reads/{A,B}.bam -s Snakefile2

Building DAG of jobs...
Job counts:
	count	jobs
	1	bwa_map
	1

[Mon Apr  8 13:42:34 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/B.fastq
    output: mapped_reads/B.bam
    jobid: 1
    wildcards: sample=B

bwa mem data/genome.fa data/samples/B.fastq | samtools view -Sb - > mapped_reads/B.bam
Job counts:
	count	jobs
	1	bwa_map
	1


In [9]:
! touch data/samples/A.fastq

In [10]:
%%bash

snakemake -np mapped_reads/{A,B}.bam -s Snakefile2

Building DAG of jobs...
Job counts:
	count	jobs
	2	bwa_map
	2

[Mon Apr  8 13:42:42 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 0
    wildcards: sample=A

bwa mem data/genome.fa data/samples/A.fastq | samtools view -Sb - > mapped_reads/A.bam

[Mon Apr  8 13:42:42 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/B.fastq
    output: mapped_reads/B.bam
    jobid: 1
    wildcards: sample=B

bwa mem data/genome.fa data/samples/B.fastq | samtools view -Sb - > mapped_reads/B.bam
Job counts:
	count	jobs
	2	bwa_map
	2


In [11]:
! cat Snakefile3

rule bwa_map:
    input:
        "data/genome.fa",
        "data/samples/{sample}.fastq"
    output:
        "mapped_reads/{sample}.bam"
    shell:
        "bwa mem {input} | samtools view -Sb - > {output}"

rule samtools_sort:
    input:
        "mapped_reads/{sample}.bam"
    output:
        "sorted_reads/{sample}.bam"
    shell:
        "samtools sort -T sorted_reads/{wildcards.sample} "
        "-O bam {input} > {output}"


In [12]:
%%bash

snakemake -np sorted_reads/B.bam -s Snakefile3

Building DAG of jobs...
Job counts:
	count	jobs
	1	bwa_map
	1	samtools_sort
	2

[Mon Apr  8 13:42:50 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/B.fastq
    output: mapped_reads/B.bam
    jobid: 1
    wildcards: sample=B

bwa mem data/genome.fa data/samples/B.fastq | samtools view -Sb - > mapped_reads/B.bam

[Mon Apr  8 13:42:50 2019]
rule samtools_sort:
    input: mapped_reads/B.bam
    output: sorted_reads/B.bam
    jobid: 0
    wildcards: sample=B

samtools sort -T sorted_reads/B -O bam mapped_reads/B.bam > sorted_reads/B.bam
Job counts:
	count	jobs
	1	bwa_map
	1	samtools_sort
	2


In [13]:
%%bash

snakemake -s Snakefile3 sorted_reads/{A,B}.bam

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	2	bwa_map
	2	samtools_sort
	4

[Mon Apr  8 13:42:53 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/B.fastq
    output: mapped_reads/B.bam
    jobid: 2
    wildcards: sample=B

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25000 sequences (2525000 bp)...
[M::mem_process_seqs] Processed 25000 reads in 1.673 CPU sec, 1.695 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem data/genome.fa data/samples/B.fastq
[main] Real time: 2.016 sec; CPU: 1.791 sec
[Mon Apr  8 13:42:55 2019]
Finished job 2.
1 of 4 steps (25%) done

[Mon Apr  8 13:42:55 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 3
    wildcards: sample=A

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25000 sequences (2525000 bp)...
[M::mem_process_seqs] Processed 25000 reads in

In [14]:
! cat Snakefile4

rule bwa_map:
    input:
        "data/genome.fa",
        "data/samples/{sample}.fastq"
    output:
        "mapped_reads/{sample}.bam"
    shell:
        "bwa mem {input} | samtools view -Sb - > {output}"

rule samtools_sort:
    input:
        "mapped_reads/{sample}.bam"
    output:
        "sorted_reads/{sample}.bam"
    shell:
        "samtools sort -T sorted_reads/{wildcards.sample} "
        "-O bam {input} > {output}"

rule samtools_index:
    input:
        "sorted_reads/{sample}.bam"
    output:
        "sorted_reads/{sample}.bam.bai"
    shell:
        "samtools index {input}"


In [15]:
%%bash

snakemake --dag sorted_reads/{A,B}.bam.bai -s Snakefile4 | dot -Tsvg > dag.svg

Building DAG of jobs...


![DAG](dag.svg)

In [21]:
%%bash

snakemake -s Snakefile4 sorted_reads/{A,B}.bam.bai

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	2	samtools_index
	2

[Mon Apr  8 13:56:33 2019]
rule samtools_index:
    input: sorted_reads/B.bam
    output: sorted_reads/B.bam.bai
    jobid: 1
    wildcards: sample=B

[Mon Apr  8 13:56:33 2019]
Finished job 1.
1 of 2 steps (50%) done

[Mon Apr  8 13:56:33 2019]
rule samtools_index:
    input: sorted_reads/A.bam
    output: sorted_reads/A.bam.bai
    jobid: 0
    wildcards: sample=A

[Mon Apr  8 13:56:33 2019]
Finished job 0.
2 of 2 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-04-08T135633.686610.snakemake.log


In [22]:
%%bash

snakemake -s Snakefile5

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	bcftools_call
	1

[Mon Apr  8 13:56:38 2019]
rule bcftools_call:
    input: data/genome.fa, sorted_reads/A.bam, sorted_reads/B.bam, sorted_reads/A.bam.bai, sorted_reads/B.bam.bai
    output: calls/all.vcf
    jobid: 0

[warning] samtools mpileup option `g` is functional, but deprecated. Please switch to using bcftools mpileup in future.
[mpileup] 2 samples in 2 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[Mon Apr  8 13:56:39 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-04-08T135637.973158.snakemake.log


In [23]:
%%bash

snakemake -s Snakefile6

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	plot_quals
	1

[Mon Apr  8 13:57:31 2019]
rule plot_quals:
    input: calls/all.vcf
    output: plots/quals.svg
    jobid: 0

[Mon Apr  8 13:57:34 2019]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2019-04-08T135731.206908.snakemake.log


In [25]:
! ls 

quals.svg


![quals](plots/quals.svg)

In [33]:
! touch data/samples/A.fastq

In [34]:
%%bash

snakemake --dag -s Snakefile-all | dot -Tsvg > dag-all.svg

Building DAG of jobs...


![dag-all](dag-all.svg)

In [35]:
%%bash 

snakemake -s Snakefile-all

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	all
	1	bcftools_call
	1	bwa_map
	1	plot_quals
	1	samtools_index
	1	samtools_sort
	6

[Mon Apr  8 14:05:01 2019]
rule bwa_map:
    input: data/genome.fa, data/samples/A.fastq
    output: mapped_reads/A.bam
    jobid: 8
    wildcards: sample=A

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 25000 sequences (2525000 bp)...
[M::mem_process_seqs] Processed 25000 reads in 1.544 CPU sec, 1.591 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem data/genome.fa data/samples/A.fastq
[main] Real time: 1.929 sec; CPU: 1.683 sec
[Mon Apr  8 14:05:03 2019]
Finished job 8.
1 of 6 steps (17%) done

[Mon Apr  8 14:05:03 2019]
rule samtools_sort:
    input: mapped_reads/A.bam
    output: sorted_reads/A.bam
    jobid: 4
    wildcards: sample=A

[Mon Apr  8 14:05:04 2019]
Finished job 4.
2 of 6 steps (33%) done

[Mon Apr  8 14:05:04 2019]
rule